In [ ]:
!pip install gradio torch diffusers transformers accelerate invisible-watermark safetensors controlnet_aux --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

# Replace 'YOUR_TOKEN_HERE' with your actual Hugging Face access token.
login(token=)

In [ ]:
# --- IMPORTS ---
import gradio as gr
import torch
from diffusers import (
    AutoPipelineForText2Image,
    AutoPipelineForImage2Image,
    AutoPipelineForInpainting,
    ControlNetModel,
    StableDiffusionXLControlNetPipeline,
    DPMSolverMultistepScheduler
)
from controlnet_aux import HEDdetector
from PIL import Image, ImageOps
import numpy as np
import time
from collections import Counter
import matplotlib.pyplot as plt
import os
import gc
import io
import zipfile
from datetime import datetime
import random

# --- CONFIGURATION & GESTION DE LA MÉMOIRE ---
print("--- Configuration & Initialisation ---")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilisation du device : {device}")

# --- IDs de modèles ---
BASE_MODEL = "stabilityai/stable-diffusion-xl-base-1.0"
REFINER_MODEL = "stabilityai/stable-diffusion-xl-refiner-1.0"
# Changement ici : on utilise un modèle ControlNet différent pour le sketch
CONTROLNET_MODEL = "diffusers/controlnet-canny-sdxl-1.0"
INPAINTING_MODEL = "diffusers/stable-diffusion-xl-1.0-inpainting-0.1"

print(f"Modèle de base : {BASE_MODEL}")
print(f"Modèle de raffinement : {REFINER_MODEL}")
print(f"Modèle ControlNet : {CONTROLNET_MODEL}")
print(f"Modèle d'Inpainting (Assistant) : {INPAINTING_MODEL}")

# --- ÉTAT PARTAGÉ ---
user_data = { "generated_images": [], "feedback_log": [], "generation_stats": { "prompts": [], "times": [], "product_types": [] } }

# --- CHARGEMENT PARESSEUX DES MODÈLES ---
pipe_text2img_base, pipe_text2img_refiner, pipe_controlnet, pipe_img2img, pipe_inpainting, hed_detector = None, None, None, None, None, None
def get_scheduler(pipe_config): return DPMSolverMultistepScheduler.from_config(pipe_config, use_karras_sigmas=True)
def load_text2img_xl_pipeline():
    global pipe_text2img_base, pipe_text2img_refiner
    if pipe_text2img_base is None:
        print("Chargement du pipeline SDXL Base...")
        pipe_text2img_base = AutoPipelineForText2Image.from_pretrained(BASE_MODEL, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to(device)
        pipe_text2img_base.scheduler = get_scheduler(pipe_text2img_base.scheduler.config)
    if pipe_text2img_refiner is None:
        print("Chargement du pipeline SDXL Refiner...")
        pipe_text2img_refiner = AutoPipelineForImage2Image.from_pretrained(REFINER_MODEL, text_encoder_2=pipe_text2img_base.text_encoder_2, vae=pipe_text2img_base.vae, torch_dtype=torch.float16, use_safetensors=True, variant="fp16").to(device)
    return pipe_text2img_base, pipe_text2img_refiner
def load_hed_detector():
    global hed_detector
    if hed_detector is None:
        print("Chargement du détecteur HED..."); hed_detector = HEDdetector.from_pretrained('lllyasviel/Annotators').to(device); print("Détecteur HED chargé.")
    return hed_detector
def load_controlnet_xl_pipeline():
    global pipe_controlnet
    if pipe_controlnet is None:
        print("Chargement ControlNet pour SDXL..."); controlnet = ControlNetModel.from_pretrained(CONTROLNET_MODEL, torch_dtype=torch.float16, use_safetensors=True).to(device); pipe_controlnet = StableDiffusionXLControlNetPipeline.from_pretrained(BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to(device); pipe_controlnet.scheduler = get_scheduler(pipe_controlnet.scheduler.config); print("Pipeline ControlNet chargé.")
    return pipe_controlnet
def load_img2img_xl_pipeline():
    global pipe_img2img
    if pipe_img2img is None:
        print("Chargement Img2Img pour SDXL..."); pipe_img2img = AutoPipelineForImage2Image.from_pretrained(BASE_MODEL, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to(device); pipe_img2img.scheduler = get_scheduler(pipe_img2img.scheduler.config); print("Pipeline Img2Img chargé.")
    return pipe_img2img
def load_inpainting_xl_pipeline():
    global pipe_inpainting
    if pipe_inpainting is None:
        print("Chargement Inpainting pour SDXL..."); pipe_inpainting = AutoPipelineForInpainting.from_pretrained(INPAINTING_MODEL, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to(device); pipe_inpainting.scheduler = get_scheduler(pipe_inpainting.scheduler.config); print("Pipeline Inpainting chargé.")
    return pipe_inpainting

# --- FONCTIONS UTILITAIRES (AVEC MODIFICATIONS) ---
def clean_memory():
    if device == "cuda": torch.cuda.empty_cache()
    gc.collect()

def preprocess_sketch_hed(sketch_dict):
    if sketch_dict is None or sketch_dict['composite'] is None:
        return Image.new('RGB', (1024, 1024), color='white')

    # On utilise l'image "composite" qui contient le fond + le dessin
    sketch_image = sketch_dict['composite'].convert("RGB")

    hed = load_hed_detector()
    sketch_np = np.array(sketch_image.convert("L"))
    if np.mean(sketch_np) > 128: # Fond majoritairement blanc
        sketch_image = ImageOps.invert(sketch_image.convert("L")).convert("RGB")

    processed_image = hed(sketch_image, detect_resolution=1024, image_resolution=1024, scribble=False)
    return processed_image

def parse_product_type(prompt):
    prompt_lower = prompt.lower()
    categories = {"Mobilier": ["sofa", "chair", "table", "desk", "bed", "shelf", "cabinet", "fauteuil", "chaise", "bureau", "lit", "étagère", "canapé", "armoire"],"Décoration": ["vase", "rug", "curtain", "painting", "mirror", "tapis", "rideau", "tableau", "miroir", "sculpture", "coussin"],"Luminaire": ["lamp", "chandelier", "light", "lampe", "lustre", "applique"],"Électroménager": ["fridge", "oven", "microwave", "kettle", "toaster", "réfrigérateur", "four", "bouilloire", "grille-pain"],"Architecture": ["building", "house", "facade", "interior", "room", "bâtiment", "maison", "façade", "intérieur", "pièce"]}
    for category, keywords in categories.items():
        if any(keyword in prompt_lower for keyword in keywords):
            return category
    return "Autre"

def estimate_dfx_metrics(cost_level, quality_level, durability_level):
    cost_multipliers = {"Low": (100, 300), "Medium": (300, 600), "High": (600, 1000)}; cost = np.random.uniform(*cost_multipliers[cost_level])
    cost_phrases = {"Low": "matériaux de base (ex: pin, tissus standards)", "Medium": "matériaux de milieu de gamme (ex: chêne, cuir synthétique de qualité)", "High": "matériaux premium (ex: bois massif noble, cuir pleine fleur)"}
    quality_phrases = {"Low": "une fabrication simple avec des finitions standards", "Medium": "un bon savoir-faire avec des finitions soignées", "High": "une fabrication de précision par des artisans, avec des finitions haut de gamme"}
    durability_phrases = {"Low": "une durée de vie estimée de 2-4 ans", "Medium": "une durée équilibrée de 5-8 ans", "High": "une durabilité exceptionnelle conçue pour durer plus de 10 ans"}
    justification = (f"**Justification détaillée :** Ce coût est une estimation basée sur l'utilisation de **{cost_phrases[cost_level]}**. "
                     f"Cela implique **{quality_phrases[quality_level]}**. En conséquence, le produit est conçu pour **{durability_phrases[durability_level]}**.")
    cost_exp = f"**Coût Estimé :** {cost:.2f} €\n{justification}"
    return cost_exp

# --- FONCTION PRINCIPALE DE GÉNÉRATION (REVUE ET CORRIGÉE) ---
def generate_images_master(prompt, seed, cost, quality, durability,
                           sketch_dict, empty_room, num_images,
                           guidance_scale, controlnet_scale,
                           img2img_strength, img2img_steps,
                           progress=gr.Progress()):
    start_time = time.time()
    if not prompt:
        raise gr.Error("Le champ 'Prompt' ne peut pas être vide.")

    enhanced_prompt = f"{prompt}, masterpiece, 8k, ultra high quality, photorealistic, professional photography, detailed"
    negative_prompt = "low quality, worst quality, blurry, distorted, watermark, signature, ugly, deformed, noisy, jpeg artifacts, cartoon, 3d render, anime, text, letters, monochrome, duplicate, error, bad anatomy"

    images = []

    use_sketch = sketch_dict is not None and sketch_dict['composite'] is not None

    try:
        current_seed = int(seed) if seed != -1 else random.randint(0, 2**32 - 1)
        generator = torch.Generator(device=device).manual_seed(current_seed)
        print(f"Utilisation du seed : {current_seed}")

        # Mode 1 : Sketch-to-Image (ControlNet)
        if use_sketch:
            progress(0.1, desc="Chargement du pipeline ControlNet...")
            pipe = load_controlnet_xl_pipeline()
            control_image = preprocess_sketch_hed(sketch_dict)
            progress(0.2, desc="Génération depuis l'esquisse (SDXL)...")
            for i in progress.tqdm(range(int(num_images)), desc="Génération ControlNet"):
                img = pipe(prompt=enhanced_prompt, negative_prompt=negative_prompt, image=control_image,
                           num_inference_steps=40, guidance_scale=float(guidance_scale),
                           controlnet_conditioning_scale=float(controlnet_scale),
                           height=1024, width=1024, generator=generator).images[0]
                images.append(img)

        # Mode 2 : Image-to-Image (Intégration dans une pièce)
        elif empty_room is not None:
            progress(0.1, desc="Chargement du pipeline Img2Img...")
            pipe = load_img2img_xl_pipeline()
            init_image = empty_room.resize((1024, 1024))
            progress(0.2, desc="Intégration dans la pièce (SDXL)...")
            for i in progress.tqdm(range(int(num_images)), desc="Génération Img2Img"):
                img = pipe(prompt=enhanced_prompt, negative_prompt=negative_prompt, image=init_image,
                           strength=float(img2img_strength), guidance_scale=float(guidance_scale),
                           num_inference_steps=int(img2img_steps), generator=generator).images[0]
                images.append(img)

        # Mode 3 : Text-to-Image (Génération pure)
        else:
            progress(0.1, desc="Chargement des pipelines SDXL Base + Refiner...")
            base, refiner = load_text2img_xl_pipeline()
            n_steps = 40; high_noise_frac = 0.8
            progress(0.2, desc="Génération depuis le texte (SDXL)...")
            for i in progress.tqdm(range(int(num_images)), desc="Génération Text2Img"):
                latents = base(prompt=enhanced_prompt, negative_prompt=negative_prompt, num_inference_steps=n_steps, denoising_end=high_noise_frac, output_type="latent", height=1024, width=1024, guidance_scale=float(guidance_scale), generator=generator).images
                img = refiner(prompt=enhanced_prompt, negative_prompt=negative_prompt, num_inference_steps=n_steps, denoising_start=high_noise_frac, image=latents, generator=generator).images[0]
                images.append(img)

        time_taken = time.time() - start_time
        cost_justification = estimate_dfx_metrics(cost, quality, durability)
        user_data["generated_images"].extend(images)
        user_data["generation_stats"]["prompts"].append(prompt)
        user_data["generation_stats"]["times"].append(time_taken / int(num_images) if int(num_images) > 0 else 0)
        user_data["generation_stats"]["product_types"].extend([parse_product_type(prompt)] * len(images))

        return images, cost_justification, f"Temps: {time_taken:.2f}s", f"Seed: {current_seed}"

    except Exception as e:
        print(f"Erreur de génération : {e}"); import traceback; traceback.print_exc(); raise gr.Error(f"Erreur de génération: {e}")
    finally:
        clean_memory()

# --- ASSISTANT DE RETOUCHE (MODIFIÉ pour ImageEditor) ---
def assistant_modify_image(image_dict, modification_prompt, strength, seed, progress=gr.Progress()):
    if image_dict is None or not modification_prompt:
        raise gr.Error("Chargez une image, dessinez un masque et entrez une requête.")

    original_image = image_dict['background'].convert("RGB").resize((1024, 1024))

    if not image_dict['layers']:
        raise gr.Error("Le masque est vide. Veuillez dessiner sur l'image la zone à modifier.")
    mask_image = image_dict['layers'][0].convert("RGB").resize((1024, 1024))

    enhanced_prompt = f"{modification_prompt}, masterpiece, 8k, photorealistic"
    negative_prompt = "low quality, blurry, watermark, signature, ugly, text, letters"

    try:
        current_seed = int(seed) if seed != -1 else random.randint(0, 2**32 - 1)
        generator = torch.Generator(device=device).manual_seed(current_seed)

        progress(0.1, desc="Chargement du pipeline d'Inpainting...")
        pipe = load_inpainting_xl_pipeline()

        progress(0.3, desc="Application de la retouche...")
        result = pipe(prompt=enhanced_prompt, negative_prompt=negative_prompt, image=original_image,
                      mask_image=mask_image, strength=float(strength), guidance_scale=8.0,
                      num_inference_steps=50, generator=generator).images[0]

        user_data["generated_images"].append(result)
        return f"Retouche appliquée. Image ajoutée à la fin de la galerie.", result

    except Exception as e:
        print(f"Erreur de retouche : {e}"); import traceback; traceback.print_exc(); raise gr.Error(f"Erreur de retouche : {e}")
    finally:
        clean_memory()

# --- Fonctions de Stats & Export (inchangées) ---
def apply_user_feedback(feedback):
    user_data["feedback_log"].append(feedback)
    return f"Feedback '{feedback}' enregistré. Total: {len(user_data['feedback_log'])} feedbacks."
def get_statistics():
    total_images, total_likes, total_dislikes = len(user_data["generated_images"]), user_data["feedback_log"].count("Like"), user_data["feedback_log"].count("Dislike")
    avg_time = np.mean(user_data["generation_stats"]["times"]) if user_data["generation_stats"]["times"] else 0
    stats_text = f"## 📊 Statistiques\n- **Images Générées:** {total_images}\n- **Feedbacks:** {total_likes} 👍, {total_dislikes} 👎\n- **Temps Moyen/Image:** {avg_time:.2f}s\n"
    fig_feedback, ax1 = plt.subplots(figsize=(5, 4)); ax1.set_title("Feedbacks");
    if total_likes > 0 or total_dislikes > 0: ax1.pie([total_likes, total_dislikes], labels=['Likes', 'Dislikes'], autopct='%1.1f%%', colors=['#4CAF50', '#F44336'], startangle=90)
    else: ax1.text(0.5, 0.5, "N/A", ha='center', va='center')
    fig_feedback.tight_layout()
    fig_types, ax2 = plt.subplots(figsize=(5, 4)); ax2.set_title("Catégories de Produits");
    if user_data["generation_stats"]["product_types"]:
        type_counts = Counter(user_data["generation_stats"]["product_types"]); ax2.pie(type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
    else: ax2.text(0.5, 0.5, "N/A", ha='center', va='center')
    fig_types.tight_layout()
    return stats_text, fig_feedback, fig_types
def export_zip():
    if not user_data["generated_images"]: raise gr.Error("Aucune image à exporter.")
    zip_io = io.BytesIO()
    with zipfile.ZipFile(zip_io, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
        for i, img in enumerate(user_data["generated_images"]):
            img_bytes = io.BytesIO(); img.save(img_bytes, format="PNG"); img_bytes.seek(0); zf.writestr(f"design_{i+1}.png", img_bytes.getvalue())
    zip_io.seek(0)
    return gr.File.from_bytes(zip_io.getvalue(), "designs.zip")

# --- INTERFACE GRAPHIQUE GRADI (MISE À JOUR AVEC ImageEditor) ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="sky")) as app:
    gr.Markdown("# 🚀 AI Design Platform v3.1 (SDXL + ImageEditor)")

    with gr.Tabs():
        with gr.Tab("🎨 Générateur de Design"):
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown("### 1. Décrivez votre idée")
                    prompt = gr.Textbox(label="Prompt", placeholder="Ex: Un fauteuil scandinave en velours bleu...")
                    gr.Markdown("### 2. Paramètres de Génération")
                    with gr.Row():
                        seed_input = gr.Number(label="Seed (-1 aléatoire)", value=-1, precision=0)
                        guidance_scale_slider = gr.Slider(1.0, 15.0, 7.5, step=0.5, label="Adhérence au Prompt")
                    num_images = gr.Slider(1, 4, value=1, step=1, label="Nombre d'images")

                    gr.Markdown("### 3. Mode de Génération (Optionnel)")
                    with gr.Tabs():
                        with gr.TabItem("✍️ Depuis un Sketch (Upload ou Dessin)"):
                            sketch_input = gr.ImageEditor(
                                type="pil",
                                label="Uploadez un sketch ou dessinez-le ici",
                                image_mode="RGB",
                                interactive=True
                            )
                            with gr.Row():
                                clear_sketch_btn = gr.Button("Effacer")
                                preview_sketch_btn = gr.Button("Prévisualiser les contours")
                            controlnet_scale_slider = gr.Slider(0.0, 2.0, 1.0, step=0.05, label="Fidélité au Sketch")
                            processed_sketch_preview = gr.Image(label="Aperçu des contours (HED)", interactive=False)

                        with gr.TabItem("🏠 Depuis une image de pièce (Img2Img)"):
                            room_input = gr.Image(type="pil", label="Importez l'image d'une pièce vide")
                            clear_room_btn = gr.Button("Effacer")
                            with gr.Row():
                                img2img_strength_slider = gr.Slider(0.1, 1.0, 0.8, step=0.05, label="Force de Modification")
                                img2img_steps_slider = gr.Slider(20, 100, 40, step=1, label="Nombre d'étapes")

                    gr.Markdown("### 4. Spécifications DFX")
                    with gr.Row():
                        cost_dd = gr.Dropdown(["Low", "Medium", "High"], value="Medium", label="Coût")
                        quality_dd = gr.Dropdown(["Low", "Medium", "High"], value="Medium", label="Qualité")
                        durability_dd = gr.Dropdown(["Low", "Medium", "High"], value="Medium", label="Durabilité")
                    gen_button = gr.Button("🚀 Générer", variant="primary", size="lg")

                with gr.Column(scale=3):
                    gr.Markdown("### ✅ Résultats (1024x1024)")
                    gallery_output = gr.Gallery(label="Designs Générés", columns=2, height="auto", object_fit="contain")
                    with gr.Row():
                        feedback_status = gr.Textbox(label="Statut Feedback", interactive=False)
                        like_button = gr.Button("👍 Like")
                        dislike_button = gr.Button("👎 Dislike")
                    seed_status = gr.Textbox(label="Seed", interactive=False)
                    with gr.Accordion("Analyse DFX", open=False):
                        dfx_output = gr.Markdown()
                    performance_output = gr.Textbox(label="Performance", interactive=False)

        with gr.Tab("🤖 Assistant IA de Retouche (Inpainting)"):
            gr.Markdown("## Modifiez un détail d'une image")
            gr.Markdown("1. Sélectionnez une image dans la galerie, elle apparaîtra ci-dessous. \n2. **Utilisez l'outil pinceau** pour masquer la zone à modifier. \n3. Décrivez la modification et lancez.")
            with gr.Row(equal_height=True):
                with gr.Column(scale=2):
                    inpainting_image_input = gr.ImageEditor(
                        type="pil",
                        label="Image à modifier",
                        interactive=True,
                        height=512
                    )
                    modification_prompt = gr.Textbox(label="Décrivez la modification", placeholder="Ex: change la couleur en rouge, ajoute des pieds en métal")
                    with gr.Row():
                        inpainting_strength_slider = gr.Slider(0.5, 1.0, 0.85, step=0.05, label="Force de retouche")
                        inpainting_seed_input = gr.Number(label="Seed", value=-1, precision=0)
                    mod_button = gr.Button("Appliquer la Retouche", variant="primary")
                with gr.Column(scale=2):
                    mod_status = gr.Textbox(label="Statut", interactive=False)
                    mod_image_output = gr.Image(label="Image Modifiée", height=512, interactive=False)

            def get_selected_image_for_editing(evt: gr.SelectData):
                if evt.value:
                    return evt.value
                return None
            gallery_output.select(fn=get_selected_image_for_editing, outputs=[inpainting_image_input])

        with gr.Tab("📊 Statistiques & Export"):
            refresh_stats_btn = gr.Button("🔄 Rafraîchir"); stats_text_output = gr.Markdown(); export_btn = gr.Button("💾 Télécharger ZIP"); zip_file_output = gr.File(label="Fichier ZIP");
            with gr.Row():
                with gr.Column(): stats_text_output; export_btn; zip_file_output; refresh_stats_btn;
                with gr.Column(): feedback_plot = gr.Plot(); types_plot = gr.Plot()

    # --- LOGIQUE DES ÉVÉNEMENTS ---
    clear_sketch_btn.click(lambda: None, outputs=[sketch_input])
    clear_room_btn.click(lambda: None, outputs=[room_input])
    preview_sketch_btn.click(fn=preprocess_sketch_hed, inputs=[sketch_input], outputs=[processed_sketch_preview])

    gen_button.click(fn=generate_images_master, inputs=[prompt, seed_input, cost_dd, quality_dd, durability_dd, sketch_input, room_input, num_images, guidance_scale_slider, controlnet_scale_slider, img2img_strength_slider, img2img_steps_slider], outputs=[gallery_output, dfx_output, performance_output, seed_status])
    like_button.click(lambda: apply_user_feedback("Like"), outputs=[feedback_status])
    dislike_button.click(lambda: apply_user_feedback("Dislike"), outputs=[feedback_status])
    mod_button.click(fn=assistant_modify_image, inputs=[inpainting_image_input, modification_prompt, inpainting_strength_slider, inpainting_seed_input], outputs=[mod_status, mod_image_output])
    refresh_stats_btn.click(fn=get_statistics, outputs=[stats_text_output, feedback_plot, types_plot])
    export_btn.click(fn=export_zip, outputs=[zip_file_output])

# --- LANCEMENT ---
if __name__ == "__main__":
    app.launch(debug=True, share=True if "COLAB_GPU" in os.environ else False)

/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segmen

--- Configuration & Initialisation ---
Utilisation du device : cuda
Modèle de base : stabilityai/stable-diffusion-xl-base-1.0
Modèle de raffinement : stabilityai/stable-diffusion-xl-refiner-1.0
Modèle ControlNet : diffusers/controlnet-canny-sdxl-1.0
Modèle d'Inpainting (Assistant) : diffusers/stable-diffusion-xl-1.0-inpainting-0.1
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5441a932f08ad52ca0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Chargement du détecteur HED...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ControlNetHED.pth:   0%|          | 0.00/29.4M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/5.14G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.fp16.saf(…):   0%|          | 0.00/167M [00:00<?, ?B/s]